## 로지스틱 회귀분석

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import numpy as np
import pandas as pd
import pingouin as pg
from scipy import stats

In [ ]:
# 시각화 관련 모듈을 호출합니다.
from GraphicSetting import *
import HelloDataScience as hds

### 실습 데이터셋 준비

In [ ]:
# 인터넷으로 공유 중인 텍스트 데이터를 읽고 데이터프레임 df를 생성합니다.
df = pd.read_csv('https://bit.ly/Univ_Admit')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

### 실습 데이터셋 전처리

In [ ]:
# rank를 문자형으로 변환합니다.
df['rank'] = df['rank'].astype('str')

In [ ]:
# 연속형 변수의 기술통계량을 확인합니다.
df.describe().round(2)

In [ ]:
# 범주형 변수의 기술통계량을 확인합니다.
df.describe(include = 'object')

In [ ]:
# rank의 원소별 빈도수를 출력합니다.
df['rank'].value_counts().sort_index()

### 목표변수 시각화

In [ ]:
# 목표변수 범주별 빈도수로 일변량 막대그래프를 그립니다.
hds.plot_bar_freq(data = df, 
                  y = 'admit', 
                  pal = ['skyblue', 'orange'])

### 연속형 입력변수와 관계 파악

In [ ]:
# admit 범주별 gre의 상자 수염 그림을 그립니다.
hds.plot_box_group(data = df, 
                   x = 'admit', 
                   y = 'gre', 
                   pal = ['skyblue', 'orange'])

In [ ]:
# admit 범주별 gpa의 상자 수염 그림을 그립니다.
hds.plot_box_group(data = df, 
                   x = 'admit', 
                   y = 'gpa', 
                   pal = ['skyblue', 'orange'])

### 범주형 입력변수와 관계 파악

In [ ]:
# rank 범주별 admit의 빈도수로 묶음 막대그래프를 그립니다.
hds.plot_bar_dodge_freq(data = df, 
                        x = 'rank', 
                        y = 'admit', 
                        pal = ['skyblue', 'orange'])

In [ ]:
# rank 범주별 admit의 빈도수로 쌓은 막대그래프를 그립니다.
hds.plot_bar_stack_freq(data = df, 
                        x = 'rank', 
                        y = 'admit', 
                        pal = ['skyblue', 'orange'])

In [ ]:
# rank 범주별 admit의 상대도수로 쌓은 막대그래프를 그립니다.
hds.plot_bar_stack_prop(data = df, 
                        x = 'rank', 
                        y = 'admit', 
                        pal = ['skyblue', 'orange'])

### t-검정: gre

In [ ]:
# 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'gre', group = 'admit')

In [ ]:
# admit 범주별 gre로 시리즈를 생성합니다.
s0 = df['gre'][df['admit'].eq('Fail')]
s1 = df['gre'][df['admit'].eq('Pass')]

In [ ]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = s0, y = s1)

### t-검정 : gpa

In [ ]:
# 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'gpa', group = 'admit')

In [ ]:
# admit 범주별 gpa로 시리즈를 생성합니다.
s0 = df['gpa'][df['admit'].eq('Fail')]
s1 = df['gpa'][df['admit'].eq('Pass')]

In [ ]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = s0, y = s1)

### 교차분석: rank

In [ ]:
# 범주형 입력변수 rank와 목표변수의 교차테이블을 출력합니다.
pd.crosstab(index = df['rank'], 
            columns = df['admit'], 
            normalize = 'index', 
            margins = True)

In [ ]:
# 교차테이블 빈도수로 교차분석(카이제곱 검정)을 실행합니다.
pg.chi2_independence(data = df, x = 'rank', y = 'admit')

### 더미변수 생성

In [ ]:
# 범주형 입력변수명으로 리스트를 생성합니다.
cols = ['admit', 'rank']

In [ ]:
# 범주형 입력변수로 더미변수를 생성합니다.
dm = pd.get_dummies(data = df[cols], drop_first = True)

In [ ]:
# df의 오른쪽에 더미변수 dm을 추가합니다.
df = pd.concat(objs = [df, dm], axis = 1)

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df에서 범주형 입력변수를 모두 삭제합니다.
df = df.drop(labels = cols, axis = 1)

In [ ]:
# 목표변수명을 'admit'으로 변경합니다.
df = df.rename(columns = {'admit_Pass': 'admit'})

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 목표변수 열이름을 지정합니다.
yvar = 'admit'

In [ ]:
# 입력변수 행렬 X와 목표변수 벡터 y를 생성합니다.
X = df.drop(labels = [yvar], axis = 1)
y = df[yvar]

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 시험셋으로 분할합니다.
Xtr, Xte, ytr, yte = train_test_split(
    X, y, test_size = 0.3, random_state = 0
)

In [ ]:
# 훈련셋의 목표변수 범주별 상대도수를 확인합니다.
ytr.value_counts(normalize = True)

In [ ]:
# 시험셋의 목표변수 범주별 상대도수를 확인합니다.
yte.value_counts(normalize = True)

### 로지스틱 회귀모형 적합

In [ ]:
# 관련 라이브러리를 호출합니다.
import statsmodels.api as sm

In [ ]:
# 로지스틱 회귀모형을 반환하는 함수를 정의합니다.
def glm(y, X):
    model = sm.GLM(endog = y, exog = X, family = sm.families.Binomial())
    return model.fit()

In [ ]:
# 훈련셋과 시험셋 입력변수에 y절편의 역할을 수행할 상수 1을 추가합니다.
Xtr = sm.add_constant(data = Xtr)
Xte = sm.add_constant(data = Xte)

In [ ]:
# 훈련셋 입력변수의 처음 5행을 출력합니다.
Xtr.head()

In [ ]:
# 훈련셋으로 로지스틱 회귀모형을 적합합니다.
fit0 = glm(y = ytr, X = Xtr)

### 로지스틱 회귀모형 적합 결과 확인

In [ ]:
# fit0 모형의 적합 결과를 확인합니다.
fit0.summary()

### 로지스틱 회귀모형의 유의성 검정

In [ ]:
# 두 모형의 이탈도 차이를 출력합니다.(검정통계량)
devGap = fit0.null_deviance - fit0.deviance
devGap

In [ ]:
# 두 모형의 자유도 차이를 출력합니다.(카이제곱 분포의 자유도)
dfGap = fit0.df_model
dfGap

In [ ]:
# 검정통계량과 자유도로 유의확률을 출력합니다.
1 - stats.chi2.cdf(x = devGap, df = dfGap)

### 다중공선성 확인

In [ ]:
# 분산팽창지수를 출력하고 다중공선성 입력변수를 확인합니다.
hds.vif(X = Xtr)

### 오즈비 확인

In [ ]:
# 입력변수별 회귀계수의 오즈비를 출력합니다.
np.exp(fit0.params).round(3)

### 표준화 회귀계수 확인

In [ ]:
# 표준화 회귀계수를 생성합니다.
# [참고] 로짓변환된 목표변수는 표준편차를 구할 수 없으므로 상수 1로 대신합니다.
beta_z = hds.std_coefs(model = fit0)

In [ ]:
# 표준화 회귀계수를 출력합니다.
beta_z

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순 정렬한 결과를 출력합니다.
beta_z.abs().sort_values()

### 목표변수의 추정확률 생성

In [ ]:
# 훈련셋의 실제값과 fit0 모형의 추정확률을 비교합니다.
ytrProb = fit0.predict(exog = Xtr)
pd.DataFrame(data = {'Real': ytr, 'Prob': ytrProb})

In [ ]:
# 시험셋의 실제값과 fit0 모형의 추정확률을 비교합니다.
yteProb = fit0.predict(exog = Xte)
pd.DataFrame(data = {'Real': yte, 'Prob': yteProb})

### 분류모형 성능 평가 : ROC 곡선

In [ ]:
# 훈련셋과 시험셋의 추정확률로 ROC 곡선을 그립니다.
hds.plot_roc(y_true = ytr, y_prob = ytrProb, color = 'red')
hds.plot_roc(y_true = yte, y_prob = yteProb, color = 'blue')

### 목표변수의 추정값 생성

In [ ]:
# 분리 기준점을 0.5로 설정합니다.
cutoff = 0.5

In [ ]:
# 훈련셋의 목표변수 추정값(라벨)을 생성합니다.
ytrPred1 = np.where(ytrProb >= cutoff, 1, 0)

In [ ]:
# 시험셋의 목표변수 추정값(라벨)을 생성합니다.
ytePred1 = np.where(yteProb >= cutoff, 1, 0)

### 분류모형 성능 평가 : 혼동행렬 리포트

In [ ]:
# 훈련셋으로 혼동행렬 리포트를 출력하고 정확도, 민감도, 정밀도 및 F1 점수를 
# 차례대로 확인합니다.
hds.clfmetrics(y_true = ytr, y_pred = ytrPred1)

In [ ]:
# 시험셋으로 혼동행렬 리포트를 출력하고 정확도, 민감도, 정밀도 및 F1 점수를 
# 차례대로 확인합니다.
hds.clfmetrics(y_true = yte, y_pred = ytePred1)

### [참고] 혼동행렬 관련 함수

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn import metrics

In [ ]:
# 시험셋의 혼동행렬을 출력합니다.
print(metrics.confusion_matrix(y_true = yte, y_pred = ytePred1))

In [ ]:
# 시험셋의 분류모형 리포트를 출력합니다.
print(metrics.classification_report(y_true = yte, y_pred = ytePred1))

In [ ]:
# 시험셋의 F1 점수를 출력합니다.
metrics.f1_score(y_true = yte, y_pred = ytePred1, pos_label = 1)

### [참고] 목표변수의 범주별 추정확률 분포

In [ ]:
# 목표변수 범주별 추정확률 분포를 확인합니다.
sns.boxplot(x = ytr, y = ytrProb)
plt.title(label = '범주별 추정확률 분포')
plt.xlabel(xlabel = '실제값')
plt.ylabel(ylabel = '추정확률')
plt.axhline(y = 0.5, color = 'red', lw = 1.5, ls = '--');

### [참고] 최적의 분리 기준점 탐색

In [ ]:
# 분리 기준점마다 분류모형의 성능지표를 계산한 데이터프레임을 생성합니다.
cfm = hds.clfCutoffs(y_true = ytr, y_prob = ytrProb)

In [ ]:
# cfm의 처음 5행을 출력합니다.
cfm.head()

In [ ]:
# 민감도와 특이도의 합계가 최댓값일 때의 분리 기준점을 확인합니다.
hds.EpiROC(obj = cfm)

### [참고] 최적의 분리 기준점으로 성능지표 확인

In [ ]:
# 최적의 분리 기준점을 설정합니다.
cutoff = 0.32

In [ ]:
# 시험셋의 목표변수 추정값(라벨)을 생성합니다.
ytePred2 = np.where(yteProb >= cutoff, 1, 0)

In [ ]:
# 혼동행렬과 F1 점수를 확인합니다.
hds.clfmetrics(y_true = yte, y_pred = ytePred2)

### [참고] 매튜의 상관계수

In [ ]:
# 분리 기준점을 다르게 설정한 추정값으로 매튜의 상관계수를 비교합니다.
metrics.matthews_corrcoef(y_true = yte, y_pred = ytePred1)

In [ ]:
# 분리 기준점을 다르게 설정한 추정값으로 매튜의 상관계수를 비교합니다.
metrics.matthews_corrcoef(y_true = yte, y_pred = ytePred2)

## End of Document